In [1]:
import pandas as pd
#!pip install xgboost
#from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import xgboost
import math
import seaborn as sns
import sklearn
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
from scipy import stats
from scipy.special import boxcox, inv_boxcox
from sklearn.model_selection import StratifiedKFold
from scipy.stats import norm
import numpy as np 

In [29]:
data = pd.read_csv("Insurance_Marketing-Customer-Value-Analysis (1).csv")
file='final-1.xlsx'
data1 = pd.read_excel(file,
sheet_name=0,
header=0,
index_col=False,
keep_default_na=True
)
data['CLV Compl']=data1['CLV Compl']
data.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,CLV Compl
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/2011,Employed,F,56274,...,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize,1650.915418
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/2011,Unemployed,F,0,...,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize,27292.856595
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/2011,Employed,F,48767,...,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize,6891.976314
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/2011,Unemployed,M,0,...,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize,40560.194784
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/2011,Employed,M,43836,...,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize,2738.670085


In [30]:
data.isnull().any()

Customer                         False
State                            False
Customer Lifetime Value          False
Response                         False
Coverage                         False
Education                        False
Effective To Date                False
EmploymentStatus                 False
Gender                           False
Income                           False
Location Code                    False
Marital Status                   False
Monthly Premium Auto             False
Months Since Last Claim          False
Months Since Policy Inception    False
Number of Open Complaints        False
Number of Policies               False
Policy Type                      False
Policy                           False
Renew Offer Type                 False
Sales Channel                    False
Total Claim Amount               False
Vehicle Class                    False
Vehicle Size                     False
CLV Compl                        False
dtype: bool

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 25 columns):
Customer                         9134 non-null object
State                            9134 non-null object
Customer Lifetime Value          9134 non-null float64
Response                         9134 non-null object
Coverage                         9134 non-null object
Education                        9134 non-null object
Effective To Date                9134 non-null object
EmploymentStatus                 9134 non-null object
Gender                           9134 non-null object
Income                           9134 non-null int64
Location Code                    9134 non-null object
Marital Status                   9134 non-null object
Monthly Premium Auto             9134 non-null int64
Months Since Last Claim          9134 non-null int64
Months Since Policy Inception    9134 non-null int64
Number of Open Complaints        9134 non-null int64
Number of Policies               

In [32]:
data.describe()

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,CLV Compl
count,9134.000000,9134.000000,9134.000000,9134.000000,9134.000000,9134.000000,9134.000000,9134.000000,9134.000000
mean,8004.940475,37657.380009,93.219291,15.097000,48.064594,0.384388,2.966170,434.088794,11318.069820
std,6870.967608,30379.904734,34.407967,10.073257,27.905991,0.910384,2.390182,290.500092,13173.262324
min,1898.007675,0.000000,61.000000,0.000000,0.000000,0.000000,1.000000,0.099007,-431.274962
25%,3994.251794,0.000000,68.000000,6.000000,24.000000,0.000000,1.000000,272.258244,3382.153078
50%,5780.182197,33889.500000,83.000000,14.000000,48.000000,0.000000,2.000000,383.945434,6547.531652
75%,8962.167041,62320.000000,109.000000,23.000000,71.000000,0.000000,4.000000,547.514839,14030.698151
max,83325.381190,99981.000000,298.000000,35.000000,99.000000,5.000000,9.000000,2893.239678,138963.030908


In [33]:
data['vehicles']=data['Vehicle Class']+"_"+['Vehicle Size']
data['loc_state']=data['State']+"_"+data['Location Code']
data=data.drop(['State','Response','Effective To Date','Gender','Location Code','Policy Type','Vehicle Class', 'Vehicle Size'],axis=1)
data.head()

,Customer,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Income,Marital Status,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,CLV Compl,vehicles,loc_state
0,BU79786,2763.519279,Basic,Bachelor,Employed,56274,Married,69,32,5,0,1,Corporate L3,Offer1,Agent,384.811147,1650.915418,Two-Door Car_Vehicle Size,Washington_Suburban
1,QZ44356,6979.535903,Extended,Bachelor,Unemployed,0,Single,94,13,42,0,8,Personal L3,Offer3,Agent,1131.464935,27292.856595,Four-Door Car_Vehicle Size,Arizona_Suburban
2,AI49188,12887.431650,Premium,Bachelor,Employed,48767,Married,108,18,38,0,2,Personal L3,Offer1,Agent,566.472247,6891.976314,Two-Door Car_Vehicle Size,Nevada_Suburban
3,WW63253,7645.861827,Basic,Bachelor,Unemployed,0,Married,106,18,65,0,7,Corporate L2,Offer1,Call Center,529.881344,40560.194784,SUV_Vehicle Size,California_Suburban
4,HB64268,2813.692575,Basic,Bachelor,Employed,43836,Single,73,12,44,0,1,Personal L1,Offer1,Agent,138.130879,2738.670085,Four-Door Car_Vehicle Size,Washington_Rural


In [34]:
dfDummies = pd.get_dummies(data['Coverage'], prefix = 'category')
data = pd.concat([data, dfDummies], axis=1)
dfDummies = pd.get_dummies(data['Education'], prefix = 'category')
data = pd.concat([data, dfDummies], axis=1)
dfDummies = pd.get_dummies(data['EmploymentStatus'], prefix = 'category')
data = pd.concat([data, dfDummies], axis=1)
dfDummies = pd.get_dummies(data['Marital Status'], prefix = 'category')
data = pd.concat([data, dfDummies], axis=1)
dfDummies = pd.get_dummies(data['Policy'], prefix = 'category')
data = pd.concat([data, dfDummies], axis=1)
dfDummies = pd.get_dummies(data['vehicles'], prefix = 'category')
data = pd.concat([data, dfDummies], axis=1)
dfDummies = pd.get_dummies(data['Renew Offer Type'], prefix = 'category')
data = pd.concat([data, dfDummies], axis=1)
dfDummies = pd.get_dummies(data['loc_state'], prefix = 'category')
data = pd.concat([data, dfDummies], axis=1)
dfDummies = pd.get_dummies(data['Sales Channel'], prefix = 'category')
data = pd.concat([data, dfDummies], axis=1)


In [35]:
data=data.drop(['Coverage','Education','EmploymentStatus','Sales Channel','Marital Status','Policy','Renew Offer Type','vehicles', 'loc_state'],axis=1)

In [39]:
#data.to_csv('file4.csv') 

In [38]:
data.columns

Index(['Customer', 'Customer Lifetime Value', 'Income', 'Monthly Premium Auto',
       'Months Since Last Claim', 'Months Since Policy Inception',
       'Number of Open Complaints', 'Number of Policies', 'Total Claim Amount',
       'CLV Compl', 'category_Basic', 'category_Extended', 'category_Premium',
       'category_Bachelor', 'category_College', 'category_Doctor',
       'category_High School or Below', 'category_Master', 'category_Disabled',
       'category_Employed', 'category_Medical Leave', 'category_Retired',
       'category_Unemployed', 'category_Divorced', 'category_Married',
       'category_Single', 'category_Corporate L1', 'category_Corporate L2',
       'category_Corporate L3', 'category_Personal L1', 'category_Personal L2',
       'category_Personal L3', 'category_Special L1', 'category_Special L2',
       'category_Special L3', 'category_Four-Door Car_Vehicle Size',
       'category_Luxury Car_Vehicle Size', 'category_Luxury SUV_Vehicle Size',
       'category_SUV_